In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sys
!{sys.executable} -m pip install pandas-profiling
!{sys.executable} -m pip install pandasql 
!{sys.executable} -m pip install impyute 

from pandasql import sqldf
import pandas_profiling
import impyute

import warnings
warnings.filterwarnings('ignore')

###############################################################################################
# Process Clinical data
###############################################################################################

patient_data = pd.read_csv('patient_data.tsv',sep='\t')
patient = pd.read_csv('patient_data.tsv',sep='\t')

patient = patient.set_index('track_name').T
patient = patient[1:]
patient = patient.reset_index()
patient = patient.rename(columns = {'index':'ID'})

patient['ID'] = patient['ID'].str.upper()
patient['Diagnosis Age'] = pd.to_numeric(patient['Diagnosis Age'])
patient['Overall Survival (Months)'] = pd.to_numeric(patient['Overall Survival (Months)'])
patient['Mutation Count'] = pd.to_numeric(patient['Mutation Count'])
patient['Sex'] = patient['Sex'].str.lower()
patient = patient[patient['# Samples per Patient']!='2'].reset_index(drop=True)

def label(x):
    if 'T1' in x:
        return 'T1'
    if 'T2' in x:
        return 'T2'
    if 'T3' in x:
        return 'T3'
    return 'T4'
patient['Stage'] = patient['American Joint Committee on Cancer Tumor Stage Code'].apply(lambda x: label(x))

patient = patient.drop(['Profiled in Mutations','Profiled in Putative copy-number alterations from GISTIC','Patient\'s Vital Status','# Samples per Patient','Cancer Type','Informed consent verified','Neoplasm Histologic Type Name'],axis=1)

###############################################################################################
# Process Tumor mRNA gene expression data
###############################################################################################

mrna_data = pd.read_csv('mrna_data.txt',sep='\t')
mrna = pd.read_csv('mrna_data.txt',sep='\t')

mrna.dropna(thresh=1,axis=1,inplace=True)
mrna = mrna.drop(['STUDY_ID'],axis=1)

def id_process(x):
    new = x[:len(x)-3]
    new = new.replace("_","")
    return new.upper()
mrna['SAMPLE_ID'] = mrna['SAMPLE_ID'].apply(lambda x: id_process(x))

###############################################################################################
# Process Tumor Mutation Sequencing data
###############################################################################################

seq_data = pd.read_csv('seq_data.txt',sep='\t')
seq = pd.read_csv('seq_data.txt',sep='\t')

seq.dropna(how='all',axis=1,inplace=True)
seq = seq.drop(['STUDY_ID'],axis=1)
seq['SAMPLE_ID'] = seq['SAMPLE_ID'].apply(lambda x: id_process(x))
seq = pd.get_dummies(seq, columns=seq.iloc[:,1:].columns,dummy_na=False)

###############################################################################################
# Process combined dataframe
###############################################################################################

def pysqldf(q):
    return sqldf(q, globals())

q = '''
SELECT *
FROM patient
JOIN mrna
ON patient.ID = mrna.SAMPLE_ID
JOIN seq
ON patient.ID = seq.SAMPLE_ID
'''
df = pysqldf(q)

df = df.drop(['SAMPLE_ID'],axis=1)

###############################################################################################
# Impute missing value

from sklearn.neighbors import KNeighborsClassifier

fitting = df
fitting = fitting.iloc[:,:90]
fitting = fitting.drop(['ID','American Joint Committee on Cancer Tumor Stage Code','Overall Survival Status','Sex','Stage'],axis=1)

race_fit = fitting.drop(['Neoplasm Histologic Grade'],axis=1)
null_index = race_fit.isnull().any(axis=1)
row_with_null = race_fit[null_index]
row_no_null = race_fit[-null_index]

clf = KNeighborsClassifier(3, weights='distance')
x = row_no_null.drop(['Race Category'],axis=1)
y = row_no_null['Race Category']
clf.fit(x,y)
x_test = row_with_null.drop(['Race Category'],axis=1)
imputes = clf.predict(x_test)

df.loc[null_index,'Race Category'] = imputes 

grade_fit = fitting.drop(['Race Category'],axis=1)
null_index = grade_fit.isnull().any(axis=1)
row_with_null = grade_fit[null_index]
row_no_null = grade_fit[-null_index]

clf = KNeighborsClassifier(3, weights='distance')
x = row_no_null.drop(['Neoplasm Histologic Grade'],axis=1)
y = row_no_null['Neoplasm Histologic Grade']
clf.fit(x,y)
x_test = row_with_null.drop(['Neoplasm Histologic Grade'],axis=1)
imputes = clf.predict(x_test)

df.loc[null_index,'Neoplasm Histologic Grade'] = imputes 


In [18]:
df_temp = df
df_temp = df.drop(['ID','American Joint Committee on Cancer Tumor Stage Code'],axis=1)

df_test = df_temp
df_test = pd.get_dummies(df_test, columns=['Sex','Race Category','Neoplasm Histologic Grade','Stage','Overall Survival Status'])

def null_to_zero(x):
    if x == 0:
        return None
    return x
df_test['Mutation Count'] = df_test['Mutation Count'].apply(lambda x: null_to_zero(x))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_test.iloc[:,:83] = scaler.fit_transform(df_test.iloc[:,:83])

In [19]:
df

,ID,American Joint Committee on Cancer Tumor Stage Code,Diagnosis Age,Overall Survival (Months),Neoplasm Histologic Grade,Race Category,Overall Survival Status,Sex,Mutation Count,Stage,...,OR51B4_R124H,OR51B4_V247L,GALNT9_P147Lfs*25,KCNK3_G117D,PCDHB1_G428R,PCDHB1_P467L,PCDHB1_R79S,LRRTM3_L138F,LRRTM3_N233K,LRRTM3_N504K
0,CG-A3-3387,T1a,49,20.27,G2,WHITE,LIVING,male,70,T1,...,0,0,0,0,0,0,0,0,0,0
1,CG-A3-3374,T1b,51,43.17,G2,BLACK OR AFRICAN AMERICAN,LIVING,female,934,T1,...,0,0,0,0,0,0,0,0,0,0
2,CG-A3-3363,T2,50,10.48,G2,ASIAN,LIVING,male,1392,T2,...,0,0,0,0,0,0,0,0,0,0
3,CG-CZ-5460,T3b,55,94.38,G2,WHITE,LIVING,male,52,T3,...,0,0,0,0,0,0,0,0,0,0
4,CG-BP-4176,T1b,64,64.22,G2,WHITE,LIVING,male,85,T1,...,0,0,0,0,0,0,0,0,0,0
5,CG-CJ-6032,T2,63,119.55,G3,WHITE,LIVING,female,42,T2,...,0,0,0,0,0,0,0,0,0,0
6,CG-BP-5192,T1a,59,23.46,G2,WHITE,LIVING,male,46,T1,...,0,0,0,0,0,0,0,0,0,0
7,CG-CW-5585,T3b,51,85.71,G2,WHITE,LIVING,male,41,T3,...,0,0,0,0,0,0,0,0,0,0
8,CG-B0-5691,T1a,66,112.71,G3,WHITE,LIVING,female,55,T1,...,0,0,0,0,0,0,0,0,0,0
9,CG-AK-3453,T2,58,83.15,G2,WHITE,LIVING,female,350,T2,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_knn = df_test.copy(deep=True)
df_em = df_test.copy(deep=True)
df_rem = df_test.copy(deep=True)

In [21]:
df_knn['Mutation Count'] = impyute.imputation.cs.fast_knn(df_test,k=5).iloc[:,2]
df_em['Mutation Count'] = impyute.imputation.cs.em(df_test).iloc[:,2]
df_rem = df_rem[-df_test['Mutation Count'].isnull()]

df_knn = df_knn.drop(['Overall Survival Status_LIVING','Overall Survival (Months)'],axis=1)
df_em = df_em.drop(['Overall Survival Status_LIVING','Overall Survival (Months)'],axis=1)
df_rem = df_rem.drop(['Overall Survival Status_LIVING','Overall Survival (Months)'],axis=1)

In [22]:
def split(df,test_sample_size):
    y = df[['Overall Survival Status_DECEASED']]
    x = df.drop(['Overall Survival Status_DECEASED'],axis=1)
    
    from sklearn import model_selection

    # Reserve 20% for testing
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=test_sample_size)
    
    print('training data has %d observation with %d features'% x_train.shape)
    print('test data has %d observation with %d features'% x_test.shape)
    
    return x_train, x_test, y_train, y_test

In [23]:
###############################################################################################
# Choose kNN for imputation based on validation results

x_train, x_test, y_train, y_test = split(df_knn,0.2)

training data has 425 observation with 201 features
test data has 107 observation with 201 features


In [24]:
###############################################################################################
# Train and select model
###############################################################################################

In [25]:
#@title build models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression

# Logistic Regression
classifier_logistic = LogisticRegression()

# K Nearest Neighbors
classifier_KNN = KNeighborsClassifier(n_neighbors=3)

# Random Forest
classifier_RF = RandomForestClassifier()

# GB Regression
classifier_GB = GradientBoostingClassifier()

In [26]:
# Train the model by logistic regression
classifier_logistic.fit(x_train, y_train)
classifier_logistic.predict(x_test)
# Accuracy of test data
classifier_logistic.score(x_test, y_test)

0.719626168224299

In [27]:
# Train the model by kNN
classifier_KNN.fit(x_train, y_train)
classifier_KNN.predict(x_test)
# Accuracy of test data
classifier_KNN.score(x_test, y_test)

0.6822429906542056

In [28]:
# Train the model by random forest
classifier_RF.fit(x_train, y_train)
classifier_RF.predict(x_test)
# Accuracy of test data
classifier_RF.score(x_test, y_test)

0.6728971962616822

In [29]:
# Train the model by gradient boosting
classifier_GB.fit(x_train, y_train)
classifier_GB.predict(x_test)
# Accuracy of test data
classifier_GB.score(x_test, y_test)

0.8037383177570093

In [31]:
# Use 10-fold Cross Validation to get the accuracy for different models
from sklearn import model_selection

model_names = ['Logistic Regression','KNN','Random Forest','Gradient Boosting']
model_list = [classifier_logistic, classifier_KNN, classifier_RF, classifier_GB]
count = 0

for classifier in model_list:
    cv_score = model_selection.cross_val_score(classifier, x_train, y_train, cv=10)
    # cprint(cv_score)
    print('Model accuracy of %s is: %.3f'%(model_names[count],cv_score.mean()))
    count += 1

Model accuracy of Logistic Regression is: 0.718
Model accuracy of KNN is: 0.638
Model accuracy of Random Forest is: 0.701
Model accuracy of Gradient Boosting is: 0.680


In [32]:
from sklearn.model_selection import GridSearchCV

# helper function for printing out grid search results 
def print_grid_search_metrics(gs):
    print ("Best score: %0.3f" % gs.best_score_)
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [33]:
# Possible hyperparamter options for Logistic Regression Regularization
# Penalty is choosed from L1 or L2
# C is the lambda value(weight) for L1 and L2
parameters = {
    'penalty':('l1', 'l2'), 
    'C':(0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.4,0.5)
}
Grid_LR = GridSearchCV(LogisticRegression(),parameters, cv=10,scoring='recall_weighted',n_jobs=-1)
Grid_LR.fit(x_train, y_train)

# the best hyperparameter combination
print_grid_search_metrics(Grid_LR)

# best LR model
best_LR_model = Grid_LR.best_estimator_

Best score: 0.741
Best parameters set:
	C: 0.15
	penalty: 'l1'


In [34]:
best_LR_model.score(x_test,y_test)

0.7289719626168224

In [ ]:
# Possible hyperparamter options for Random Forest
# Choose the number of trees
parameters = {
    'n_estimators' : [50,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,90,100],
    'min_samples_split': [3, 5, 10, 15, 20], 
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20, 30 ,50]
}
Grid_RF = GridSearchCV(RandomForestClassifier(),parameters, cv=10,scoring='recall_weighted',n_jobs=-1)
Grid_RF.fit(x_train, y_train)

# best number of tress
print_grid_search_metrics(Grid_RF)

# best random forest
best_RF_model = Grid_RF.best_estimator_

In [ ]:
# Possible hyperparamter options for KNN
# Choose k
parameters = {
    'n_neighbors':[1,2,3,4,5,6,7,8,9,10,15,20,25,30,50] 
}
Grid_KNN = GridSearchCV(KNeighborsClassifier(),parameters, cv=10, scoring='recall_weighted',n_jobs=-1)
Grid_KNN.fit(x_train, y_train)

# best k
print_grid_search_metrics(Grid_KNN)

# best KNN model
best_KNN_model = Grid_KNN.best_estimator_

In [35]:
# Possible hyperparamter options for Gradient Boosting Decision Tree
# Choose the number of trees
parameters = {
    'n_estimators' : (30,35,40,45,50,55,60,65,70,75,80),
    'learning_rate' : (0.05,0.1,0.15,0.2,0.3),
    'subsample' : (0.5,0.6,0.7,0.8,0.9,1),
    'max_depth': [3, 5, 10, 15, 20, 25],
    'max_features': [3, 5, 10, 20, 30, 50]
}
Grid_GB = GridSearchCV(GradientBoostingClassifier(),parameters, cv=10,scoring='recall_weighted',n_jobs=-1)
Grid_GB.fit(x_train, y_train)

# best number of tress
print_grid_search_metrics(Grid_GB)

# best Gradient Boosting Decision Tree
best_GB_model = Grid_GB.best_estimator_


Best score: 0.755
Best parameters set:
	learning_rate: 0.05
	max_depth: 15
	max_features: 20
	n_estimators: 80
	subsample: 1


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# calculate accuracy, precision and recall
def cal_evaluation(classifier, cm):
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    print (classifier)
    print ("Accuracy is: %0.3f" % accuracy)
    print ("precision is: %0.3f" % precision)
    print ("recall is: %0.3f" % recall)

# print out confusion matrices
def draw_confusion_matrices(confusion_matricies):
    class_names = ['DEAD','LIVING']
    for cm in confusion_matrices:
        classifier, cm = cm[0], cm[1]
        cal_evaluation(classifier, cm)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(cm, interpolation='nearest',cmap=plt.get_cmap('Reds'))
        plt.title('Confusion matrix for %s' % classifier)
        fig.colorbar(cax)
        ax.set_xticklabels([''] + class_names)
        ax.set_yticklabels([''] + class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()
        
%matplotlib inline

# Confusion matrix, accuracy, precison and recall for random forest and logistic regression
confusion_matrices = [
    ("Logistic Regression", confusion_matrix(y_test,best_LR_model.predict(x_test))),
    ("Random Forest", confusion_matrix(y_test,best_RF_model.predict(x_test))),
    ("Gradient Boosting", confusion_matrix(y_test,best_GB_model.predict(x_test)))
    
]

draw_confusion_matrices(confusion_matrices)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics

# Use predict_proba to get the probability results of Logistic Regression
y_pred_lr = best_LR_model.predict_proba(x_test)[:, 1]
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_lr)
# ROC Curve
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='lr')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - LR Model')
plt.legend(loc='best')
plt.show()

In [ ]:
# Use predict_proba to get the probability results of Random Forest
y_pred_rf = best_RF_model.predict_proba(x_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)
# ROC Curve
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='rf')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - RF Model')
plt.legend(loc='best')
plt.show()

In [ ]:
# Use predict_proba to get the probability results of Gradient Boosting model
y_pred_gb = best_GB_model.predict_proba(x_test)[:, 1]
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_pred_gb)

# ROC curve of Gradient Boosting result
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_gb, tpr_gb, label='gb')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - GB model')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_lr) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = metrics.auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_rf) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = metrics.auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_gb) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = metrics.auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
def threshold_performance(classifier, threshold):
    cm = confusion_matrix(y_test,(classifier.predict_proba(x_test)[:,1] >= threshold).astype(int))
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    print (classifier)
    print ("Threshold is: %0.3f" % threshold)
    print ("Accuracy is: %0.3f" % accuracy)
    print ("precision is: %0.3f" % precision)
    print ("recall is: %0.3f" % recall)

In [ ]:
deceased_rate = (y_train.sum()/len(y_train))[0]

In [ ]:
threshold_performance(best_LR_model, deceased_rate)

In [ ]:
threshold_performance(best_RF_model, deceased_rate)

In [ ]:
threshold_performance(best_GB_model, deceased_rate)

In [ ]:
x = df_knn.drop(['Overall Survival Status_DECEASED'],axis=1)
y = df_knn['Overall Survival Status_DECEASED']

LRmodel_l1 = best_LR_model
LRmodel_l1.fit(x, y)
LRmodel_l1.coef_[0]
print ("Logistic Regression (L1) Coefficients")
for k,v in sorted(zip(map(lambda x: round(x, 4), LRmodel_l1.coef_[0]), \
                      x.columns), key=lambda k_v:(-abs(k_v[0]),k_v[1])):
    print (v + ": " + str(k))

In [ ]:
# check feature importance of random forest for feature selection
x_feature_importance = df_knn.drop(['Overall Survival Status_DECEASED'],axis=1)
y = df_knn['Overall Survival Status_DECEASED']

forest = RandomForestClassifier()
forest.fit(x_feature_importance, y)

importances = forest.feature_importances_

# Print the feature ranking
print("Feature importance ranking by Random Forest Model:")
for k,v in sorted(zip(map(lambda x: round(x, 4), importances), x_feature_importance.columns), reverse=True):
    print (v + ": " + str(k))